In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('CSV Files/9_churnML.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
X = df.drop(columns=['CustomerId', 'Surname', 'Exited'])
y = df['Exited']

In [6]:
categorical_cols = ['Geography', 'Gender']
binary_cols = ['HasCrCard', 'IsActiveMember', 'NumOfProducts']
scale_cols = ['CreditScore', 'Balance', 'EstimatedSalary']
bin_cols = ['Age']

In [7]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [8]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), categorical_cols),
        ("scale", StandardScaler(), scale_cols),
        ("bin_age", KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='kmeans'), bin_cols),
        ('binary', 'passthrough', binary_cols),
    ]
)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [11]:
import torch 
import numpy as np

In [12]:
X_trainTensor = torch.tensor(X_train, dtype=torch.float32)
y_trainTensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_testTensor = torch.tensor(X_test, dtype=torch.float32)
y_testTensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [13]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_trainTensor, y_trainTensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [14]:
import torch.nn as nn

class ANN(nn.Module):
    def __init__(self, input_dim):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
model = ANN(X_train.shape[1])

In [15]:
import torch.optim as optim

criterion = nn.BCELoss()  # binary cross-entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [16]:
epochs = 20

for epoch in range(epochs):
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/20], Loss: 0.4904
Epoch [2/20], Loss: 0.3543
Epoch [3/20], Loss: 0.3872
Epoch [4/20], Loss: 0.2835
Epoch [5/20], Loss: 0.3408
Epoch [6/20], Loss: 0.4421
Epoch [7/20], Loss: 0.2504
Epoch [8/20], Loss: 0.3280
Epoch [9/20], Loss: 0.3136
Epoch [10/20], Loss: 0.3888
Epoch [11/20], Loss: 0.3307
Epoch [12/20], Loss: 0.4029
Epoch [13/20], Loss: 0.2079
Epoch [14/20], Loss: 0.2425
Epoch [15/20], Loss: 0.3229
Epoch [16/20], Loss: 0.4883
Epoch [17/20], Loss: 0.2182
Epoch [18/20], Loss: 0.4606
Epoch [19/20], Loss: 0.4569
Epoch [20/20], Loss: 0.2760


In [17]:
with torch.no_grad():
    y_pred = model(X_testTensor)
    y_pred_cls = (y_pred >= 0.5).float()
    
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    acc = accuracy_score(y_testTensor, y_pred_cls)
    prec = precision_score(y_testTensor, y_pred_cls)
    rec = recall_score(y_testTensor, y_pred_cls)
    f1 = f1_score(y_testTensor, y_pred_cls)
    
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")


Accuracy: 0.8335
Precision: 0.7056
Recall: 0.3120
F1 Score: 0.4327


In [19]:
num_pos = y_trainTensor.sum().item()
num_neg = y_trainTensor.shape[0] - num_pos
pos_weight = torch.tensor([num_neg / num_pos], dtype=torch.float32)
print(f"Positive class weight: {pos_weight.item():.2f}")

Positive class weight: 3.91


In [20]:
class ImprovedANN(nn.Module):
    def __init__(self, input_dim):
        super(ImprovedANN, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 32)
        self.hidden2 = nn.Linear(32, 16)
        self.output = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
    def forward(self, x):
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.hidden2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.output(x)
        return x

In [23]:
model = ImprovedANN(X_train.shape[1])

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [25]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_trainTensor, y_trainTensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

epochs = 30

for epoch in range(epochs):
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(train_loader):.4f}")


Epoch [1/30], Loss: 1.0310
Epoch [2/30], Loss: 0.9635
Epoch [3/30], Loss: 0.9406
Epoch [4/30], Loss: 0.9333
Epoch [5/30], Loss: 0.9241
Epoch [6/30], Loss: 0.9155
Epoch [7/30], Loss: 0.9011
Epoch [8/30], Loss: 0.8916
Epoch [9/30], Loss: 0.8864
Epoch [10/30], Loss: 0.8793
Epoch [11/30], Loss: 0.8630
Epoch [12/30], Loss: 0.8469
Epoch [13/30], Loss: 0.8372
Epoch [14/30], Loss: 0.8254
Epoch [15/30], Loss: 0.8175
Epoch [16/30], Loss: 0.7995
Epoch [17/30], Loss: 0.7984
Epoch [18/30], Loss: 0.7988
Epoch [19/30], Loss: 0.7913
Epoch [20/30], Loss: 0.7926
Epoch [21/30], Loss: 0.7817
Epoch [22/30], Loss: 0.7763
Epoch [23/30], Loss: 0.7826
Epoch [24/30], Loss: 0.7724
Epoch [25/30], Loss: 0.7775
Epoch [26/30], Loss: 0.7752
Epoch [27/30], Loss: 0.7721
Epoch [28/30], Loss: 0.7708
Epoch [29/30], Loss: 0.7639
Epoch [30/30], Loss: 0.7737


In [27]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

with torch.no_grad():
    logits = model(X_testTensor)
    probs = torch.sigmoid(logits)  # convert logits to probabilities
    
    # Optional: change threshold to improve recall
    threshold = 0.4
    y_pred_cls = (probs >= threshold).float()
    
    acc = accuracy_score(y_testTensor, y_pred_cls)
    prec = precision_score(y_testTensor, y_pred_cls)
    rec = recall_score(y_testTensor, y_pred_cls)
    f1 = f1_score(y_testTensor, y_pred_cls)
    
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")


Accuracy: 0.6935
Precision: 0.3850
Recall: 0.8477
F1 Score: 0.5295
